In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from torch import optim


In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df=pd.read_csv('fashion-mnist_train.csv.zip')

In [ ]:
df2=pd.read_csv('fashion-mnist_test.csv.zip')

In [ ]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.describe()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.00000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.00000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.00000
mean,4.500000,0.000900,0.006150,0.035333,0.101933,0.247967,0.411467,0.805767,2.198283,5.682000,14.488767,32.830583,45.731017,52.064083,50.80245,50.459150,53.796217,51.396233,40.432733,26.083333,10.420583,4.079750,1.938183,1.177650,0.857983,0.461133,0.244300,0.092667,0.018333,0.006400,0.020617,0.098967,0.357200,0.767217,2.104583,7.331483,17.472317,32.009017,52.814583,79.349917,...,92.470233,90.523350,79.255767,62.576500,45.753283,43.970183,47.772400,38.841583,22.306933,9.280517,3.955417,0.631000,0.024633,0.218117,1.027250,4.352033,12.568183,20.847967,21.076467,17.152267,22.114933,30.559350,41.743617,51.506583,52.60855,46.479783,42.612733,45.309733,50.162100,46.438617,34.625400,23.300683,16.588267,17.869433,22.814817,17.911483,8.520633,2.753300,0.855517,0.07025
std,2.872305,0.094689,0.271011,1.222324,2.452871,4.306912,5.836188,8.215169,14.093378,23.819481,38.334549,57.633860,65.964300,68.603587,67.86885,67.370188,69.467796,67.912739,62.639596,51.818389,32.591095,20.118496,13.953210,10.718718,9.256114,6.768024,4.953103,2.850576,1.232175,0.548966,1.184869,2.912779,6.523878,9.702476,15.977721,30.056073,47.953585,65.625544,82.122567,95.340204,...,100.912013,99.731484,95.172029,88.844198,78.434831,76.970849,83.212136,74.477031,55.983447,37.937984,24.953295,8.146274,0.968474,4.444904,10.030021,20.912278,36.920905,48.909477,48.263137,42.173000,48.241135,54.902062,61.963606,68.331468,69.76041,67.476669,66.267004,66.737520,68.705817,65.944847,57.545242,48.854427,41.979611,43.966032,51.830477,45.149388,29.614859,17.397652,9.356960,2.12587
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2

In [ ]:
y_train=df['label']

In [ ]:
x_train=df.drop('label',axis=1)

In [ ]:
y_test=df2['label']
x_test=df2.drop('label',axis=1)

In [ ]:
y_test=y_test.values

In [ ]:
x_train=x_train.values

In [ ]:
x_test=x_test.values

In [ ]:
x_train_tensor=torch.tensor(x_train).float()
y_train_tensor=torch.tensor(y_train).float()

In [ ]:
x_test_tensor=torch.tensor(x_test).float()
y_test_tensor=torch.tensor(y_test).float()

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features=features
    self.labels=labels
  def __len__(self):
    return len(self.features)
  def __getitem__(self,index):
    return self.features[index],self.labels[index]


In [ ]:
train_dataset=CustomDataset(x_train_tensor,y_train_tensor)
len(train_dataset)

60000

In [ ]:
test_dataset=CustomDataset(x_test,y_test)
len(test_dataset)

10000

In [ ]:
train_dataloader=DataLoader(train_dataset,batch_size=600,shuffle=True,pin_memory=True)
test_dataloader= DataLoader(test_dataset,batch_size=100,shuffle=False,pin_memory=True)
print(len(train_dataloader))
print(len(test_dataloader))


100
100


In [ ]:
class NeuralNet(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.features=num_features
    self.architecture=nn.Sequential(
        nn.Linear(self.features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )
  def forward(self,x):
    return self.architecture(x)

In [ ]:
x_train_tensor.shape[0]

60000

In [ ]:
model=NeuralNet(x_train_tensor.shape[1]).to(device)
loss=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)


In [ ]:
epochs=100
y_pred_train=[]
for i in range(epochs):
  total_loss=0
  for batch_features,batch_labels in train_dataloader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    batch_y_pred=model(batch_features)
    batch_loss=loss(batch_y_pred,batch_labels.long())
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    total_loss+=batch_loss.item()
    if i==99:
        y_pred_train.append(batch_y_pred.detach().cpu())
  avg_loss=total_loss/len(train_dataloader)
  if i%10==0:
    print(f'epoch {i+1} : avg_loss={avg_loss}')
y_pred_tensor=torch.cat(y_pred_train,dim=0)



epoch 1 : avg_loss=12.12050011396408
epoch 11 : avg_loss=0.44099038690328596
epoch 21 : avg_loss=0.39739041060209274
epoch 31 : avg_loss=0.38220628947019575
epoch 41 : avg_loss=0.4023210522532463
epoch 51 : avg_loss=0.38452518582344053
epoch 61 : avg_loss=0.39349168866872786
epoch 71 : avg_loss=0.4046530631184578
epoch 81 : avg_loss=0.38868658035993575
epoch 91 : avg_loss=0.37381196349859236


In [ ]:
y_pred_tensor.shape

torch.Size([60000, 10])

In [ ]:
model.eval()

NeuralNet(
  (architecture): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
batch_y_pred.shape

torch.Size([600, 10])

In [ ]:
a=torch.tensor([[1,2],[2,3]]).float()
b=torch.tensor([[1,2],[2,4]]).float()

print((a==b).sum())

tensor(3)


In [ ]:
for batch_features, batch_labels in test_dataloader:
  print(batch_labels.shape) #1d
  break

torch.Size([100])


In [ ]:
total=0
correct=0
y_pred_test=[]
with torch.no_grad():
  for batch_features, batch_labels in test_dataloader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    batch_y_pred_test=model(batch_features.float())
    output=torch.argmax(batch_y_pred_test,dim=1)
    total = total + batch_labels.shape[0]
    correct = correct + (output == batch_labels).sum().item()
    y_pred_test.append(output.detach().cpu())
  print(f"Accuracy of test dataset is {correct/total}")
y_pred_test_tensor=torch.cat(y_pred_test,dim=0)


Accuracy of test dataset is 0.8555


In [ ]:
y_pred_test_tensor.shape

torch.Size([10000])

In [ ]:
output.shape

torch.Size([100])

In [ ]:
y_pred_test

[tensor([0, 1, 2, 2, 3, 6, 8, 6, 5, 0, 3, 4, 6, 6, 8, 5, 6, 3, 6, 4, 4, 4, 2, 1,
         5, 7, 8, 4, 4, 1, 5, 7, 7, 8, 1, 0, 9, 6, 6, 8, 2, 0, 4, 2, 2, 0, 3, 3,
         2, 3, 2, 4, 9, 3, 0, 9, 9, 4, 6, 0, 4, 5, 4, 6, 1, 1, 0, 9, 5, 2, 7, 3,
         4, 0, 5, 7, 1, 6, 1, 6, 9, 8, 1, 2, 4, 8, 9, 4, 1, 6, 3, 4, 4, 2, 6, 0,
         4, 7, 7, 3]),
 tensor([9, 3, 9, 6, 8, 2, 3, 8, 2, 7, 5, 5, 3, 2, 7, 5, 6, 2, 7, 1, 0, 5, 4, 4,
         7, 0, 5, 5, 0, 3, 1, 7, 9, 4, 9, 6, 4, 6, 6, 4, 3, 3, 0, 2, 2, 6, 3, 0,
         1, 3, 4, 3, 3, 1, 9, 0, 3, 3, 9, 5, 6, 7, 7, 3, 2, 4, 0, 2, 7, 2, 6, 8,
         9, 0, 2, 4, 2, 5, 7, 9, 7, 1, 3, 9, 1, 8, 5, 6, 0, 7, 4, 9, 1, 6, 0, 0,
         6, 4, 6, 9]),
 tensor([0, 4, 4, 5, 5, 8, 0, 2, 1, 9, 2, 4, 7, 1, 9, 5, 9, 6, 2, 8, 5, 7, 7, 0,
         2, 4, 5, 7, 8, 1, 9, 7, 0, 2, 9, 7, 4, 0, 9, 2, 1, 5, 7, 7, 0, 2, 4, 5,
         3, 3, 8, 1, 6, 4, 4, 8, 4, 9, 8, 3, 3, 2, 4, 4, 2, 9, 3, 9, 2, 8, 6, 8,
         8, 2, 7, 2, 5, 5, 9, 7, 8, 4, 4, 8, 3, 8, 3, 1, 2, 2, 